In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [2]:
all_indicators_df = pd.read_csv('body/wmd_all_indicators.csv')

In [3]:
not_arg_df = all_indicators_df[all_indicators_df['label'] == 'nao_argumento']
arg_df = all_indicators_df[all_indicators_df['label'] != 'nao_argumento']

not_arg_rates = not_arg_df['wmd_rate']
arg_rates = arg_df['wmd_rate']


In [4]:
# fig, ax = plt.subplots(figsize=(9, 9))
# ax.boxplot([arg_rates, not_arg_rates], labels=['Argumento', 'Não argumento'], showmeans=True)

# ax.set_yticks(np.arange(0.7, 0.81, 0.006))

# plt.grid(True, axis='y')
# plt.show()

In [5]:
f = px.box(all_indicators_df, y='wmd_rate', x='label', title="Nota do WMD por tipos de sentenças")
f.update_yaxes(
    dtick=0.009
)
f.show()

In [6]:
grouped_by_arg = all_indicators_df.copy()
grouped_by_arg['label'] = all_indicators_df['label'].apply(lambda x: x if x == 'nao_argumento' else 'argumento')

In [7]:
f = px.box(grouped_by_arg, y='wmd_rate', color='label', notched=True, color_discrete_sequence=['forestgreen', 'indianred'], title="Nota do WMD - Argumento x Não argumento", height=610)
f.update_yaxes(
    dtick=0.006
)
f.show()

### boxplot sem utilizar unigramas nos indicadores

In [8]:
wmd_u0 = pd.read_csv('body/without_unigrams/wmd_u0_all_indicators.csv')

In [9]:
f = px.box(wmd_u0, y='wmd_rate', x='label', notched=True, title="Nota do WMD por tipo de sentenças (sem unigramas)", height=600)
f.update_yaxes(
    dtick=0.006
)
f.show()

# Histograma
Tamanho das redações: quantidade de palavras

In [10]:
all_indicators_df['len_text'] = all_indicators_df['text'].apply(lambda x: len(x.split()))
grouped_by_arg['len_text'] = grouped_by_arg['text'].apply(lambda x: len(x.split()))

In [11]:
f = px.box(all_indicators_df, y='len_text', x='label', notched=True, title="Comparação da quantidade de palavras por sentença em cada categoria", height=550, points="all")
f.update_coloraxes()
f.update_yaxes(
    dtick=5,
    title='palavras por sentença'
)
f.show()

In [12]:
fig = px.histogram(all_indicators_df, x='len_text', color="label", nbins=20, labels={"len_text": "quantidade de palavras"}, title="Histograma de quantidade de palavras dos trechos de redação", barmode='group')
fig.update_xaxes(
    showgrid=True,
    dtick = 5
)
fig.show()

In [13]:
fig = px.histogram(grouped_by_arg, x='len_text', color="label", labels={"len_text": "quantidade de palavras"}, color_discrete_sequence=["limegreen", "indianred"], nbins=20, barmode='group', title="Comparativo entre redações marcadas como argumento e não argumento")

fig.update_xaxes(
    showgrid=True,
    dtick = 5
)
fig.show()

## Boxplot com cada uma das fontes

In [14]:
source_names = [
    'brasil_escola',
    'clue_words_pt',
    'acrobata_das_letras',
    'mundo_educacao',
    'Subjetivity_lexicon',
    'PersuativeEssays_UKP',
    '2020_Jonathan'
]

sources = list()
for name in source_names:
    df = pd.read_csv(f'body/wmd_{name}.csv')
    df['source'] = name
    sources.append(df)
df = pd.concat(sources)

In [15]:
not_arg_source_df = df[df['label'] == 'nao_argumento']
arg_source_df = df[df['label'] != 'nao_argumento']

In [16]:
fig = go.Figure()

fig.add_trace(go.Box(
    y=not_arg_source_df['wmd_rate'],
    x=not_arg_source_df['source'],
    name="Não argumento",
    marker_color='indianred'
))
fig.add_trace(go.Box(
    y=arg_source_df['wmd_rate'],
    x=arg_source_df['source'],
    name="Argumento",
    marker_color='forestgreen'

)) 
fig.update_layout(
    boxmode='group',
    title="WMD em comparação com as fontes de indicadores",
    height=650,
    width=900,
    yaxis=dict(dtick=0.009, title="Nota do WMD")

)

## histograma quantidade de indicadores por fonte

In [17]:
amount_indicators = dict()
for name in source_names:
    folder = f"Indicadores/{name}"
    words = list()
    for file_name in os.listdir(folder):
        file_path = os.path.join(folder, file_name)
        with open(file_path) as f:
            words.extend([w.rstrip() for w in f.readlines()])
    amount_indicators[name] = len(words)

len_all_indicators = len(open('Indicadores/preprocessed_indicators.txt').readlines())
amount_indicators['all_indicators'] = len_all_indicators

In [18]:
barcolors=['lightsalmon']*7 + ['indianred']
x = list(amount_indicators.keys())
y = list(amount_indicators.values())
fig = go.Figure(data=go.Bar(
    y=y,
    x=x,
    text=y,
    textposition='auto',
    marker_color=barcolors
))

fig.update_layout(
    title="Quantidade de indicadores por fonte",
    xaxis={
        'categoryorder':'array',
        'categoryarray': [xi for _,xi in sorted(zip(y, x))]
        }
)
fig.show()

## Limitando as sentenças

Sentenças entre 20 e 30 palavras

In [34]:
def show_ranged_boxplot(init_n_word, final_n_word):
    reduced_all_df = all_indicators_df[all_indicators_df['len_text'].ge(init_n_word) & all_indicators_df['len_text'].le(final_n_word)]
    reduced_pair_df = grouped_by_arg[grouped_by_arg['len_text'].ge(init_n_word) & grouped_by_arg['len_text'].le(final_n_word)]
    vc_pair = reduced_pair_df['label'].value_counts()
    vc_all = reduced_all_df['label'].value_counts()
    amount_arg_all = vc_pair['argumento']
    amount_not_arg = vc_pair['nao_argumento']
    space_20=" "*20
    space_15=" "*15
    amount_tese = vc_all['tese']
    amount_pi = vc_all['PI']
    amount_arg = vc_all['argumento']

    f = make_subplots(rows=1, cols=2, column_titles=['agrupado','desagrupado'], column_widths=[0.4, 0.6])
    f.add_trace(go.Box(
        marker_color='palevioletred',
        x=reduced_pair_df['label'],
        y=reduced_pair_df['wmd_rate'],
        boxpoints='all',
        name="agrupados"
    ), row=1, col=1)
    f.add_trace(go.Box(
        marker_color='forestgreen',
        x=reduced_all_df['label'],
        y=reduced_all_df['wmd_rate'],
        boxpoints='all',
        name="desagrupados"
    ), row=1, col=2)
    f.update_layout(
        title=f"Nota do WMD por tipo de sentenças de {init_n_word} a {final_n_word} palavras",
        height=550,
        width=800,
        boxmode='group',
        showlegend=False
    )
    f.update_yaxes(dtick=0.006)
    f.update_xaxes(
        title=f'{amount_arg_all}{space_20}{amount_not_arg}',
        categoryorder = 'array',
        categoryarray =  ['argumento', 'nao_argumento'],
        row=1,
        col=1)
    f.update_xaxes(
        title=f'{amount_pi}{space_15}{amount_arg}{space_15}{amount_tese}{space_15}{amount_not_arg}',
        categoryorder = 'array',
        categoryarray =  ['PI', 'argumento', 'tese', 'nao_argumento'],
        row=1,
        col=2
    )

    return f.show()

In [50]:
max_len = 67
ranges=[
    (0, 6),
    (0, 10),
    (10, max_len),
    (15, max_len),
    (20, max_len),
    (25, max_len),
    (20, 30),
    (25, 30),
    (25, 35),
    (15, 20),
    (10, 20),
    (10, 30)
]
# 10,15 == 10,20 == 15,20
for init, final in ranges:
    show_ranged_boxplot(init, final)
    

## Debugando os textos

A seguir, é mostrado as 10 primeiras sentenças de tamanho 6.
Prestar atenção na linha do index **189**

In [86]:
with pd.option_context('display.max_colwidth', None):
    display(all_indicators_df[all_indicators_df['len_text'] == 6].head(10))

,id,text,label,wmd_rate,len_text
44,8,presos modo consumir baseado consumimos gostamos,argumento,0.757395,6
49,9,necessário preocupação pais conteúdo oferecido filho,PI,0.775413,6
54,10,internet seleciona bens serviços consumidos população,argumento,0.792499,6
145,27,brasil questões voltadas sexo ainda tabus,argumento,0.772552,6
148,27,políticas combate existentes sensibilizando jovens deveriam,argumento,0.783191,6
178,31,modernidade líquida fato responsável permanência suicídio,argumento,0.771876,6
189,32,significância bem-estar psicológico portanto deve menosprezada,argumento,0.723772,6
208,34,observa-se sociedade impõe inúmeras exigências jovens,argumento,0.770033,6
220,35,muitos casos violência ignorados pessoas assistem,argumento,0.777144,6
239,37,hoax notícias falsas mudam pensamento leitor,argumento,0.771891,6


In [92]:
with pd.option_context('display.max_colwidth', None):
    display(all_indicators_df[all_indicators_df['len_text'].eq(10) & all_indicators_df['wmd_rate'].lt(0.75)].sort_values(by=['wmd_rate']))

,id,text,label,wmd_rate,len_text
249,38,reforçam algum ideal político ajudando fortalecer crenças geralmente apelativas emocionalmente,argumento,0.730298,10
428,142,assim insatisfação poeta heterônimos poderiam continuar menor preocupação permanecermos miopia,nao_argumento,0.732683,10
480,290,coadunação tal preceito urge necessidade medidas eficazes evitar inércia impasses,nao_argumento,0.734629,10
222,35,muitos casos suicídio poderiam evitados caso conhecimento situação indivíduo encontra,argumento,0.738816,10
296,43,racismo ainda existe país muitos passos devem dados alcançar atenuaçã,tese,0.739512,10
553,535,tudo circula redes sociais verdadeiro embora atualmente principais veículos informação,nao_argumento,0.740824,10
442,169,evidente portanto medidas tomadas direito liberdade escolha indivíduo fato respeitado,nao_argumento,0.740969,10
174,30,grande número portadores devido conhecimento doença reforça ainda importância informação,argumento,0.741924,10
419,130,dias hoje porém inteligência artificial baseada códigos binários domina mundo,nao_argumento,0.742589,10
424,137,assim fácil compreensão exito famosas fake news diferentes campos afeta,nao_argumento,0.746267,10
